In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext('local[2]', 'pyspark matrix mult')

In [3]:
def index_col(x):
    cnt = -1
    for n in x:
        cnt+=1
        yield (cnt, n)

In [4]:
A = sc.textFile('mat_A.txt').map(lambda x: x.split(';')).map(lambda x: (int(x[0]),eval(x[1])))
B = sc.textFile('vec_B.txt').map(eval)

B = B.flatMap(index_col)

A = A.flatMapValues(index_col).map(lambda x: (x[1][0],(x[0],x[1][1])))
W = A.join(B)

W = W.map(lambda x: (x[1][0][0], x[1][0][1]*x[1][1]))

W = W.reduceByKey(lambda x,y: x+y)

W.collect()

W.saveAsTextFile('multi_matrix.txt')